# Визуализация орбиты TESS

TESS (Transiting Exoplanet Survey Satellite) имеет уникальную орбиту:
- **Резонанс 2:1 с Луной** — за 2 оборота Луны TESS делает 1 оборот
- **Период**: ~13.7 дней
- **Перигей**: ~108,000 км (ближайшая точка к Земле)
- **Апогей**: ~373,000 км (дальняя точка)

Данные собираются в основном около апогея, когда телескоп стабилен.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Ellipse, FancyArrowPatch
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import warnings
warnings.filterwarnings('ignore')

print("Imports OK")

## Параметры орбит

In [ ]:
# Масштаб: 1 единица = 10,000 км
SCALE = 10000  # км на единицу

# Земля
EARTH_RADIUS = 6371 / SCALE  # км -> единицы

# Луна
MOON_DISTANCE = 384400 / SCALE  # км -> единицы
MOON_RADIUS = 1737 / SCALE
MOON_PERIOD = 27.3  # дней

# TESS орбита (эллиптическая)
TESS_PERIGEE = 108000 / SCALE  # км -> единицы
TESS_APOGEE = 373000 / SCALE
TESS_PERIOD = 13.7  # дней

# Эллипс TESS
TESS_a = (TESS_PERIGEE + TESS_APOGEE) / 2  # большая полуось
TESS_c = TESS_a - TESS_PERIGEE  # фокусное расстояние
TESS_b = np.sqrt(TESS_a**2 - TESS_c**2)  # малая полуось
TESS_e = TESS_c / TESS_a  # эксцентриситет

print(f"Земля: радиус {EARTH_RADIUS:.2f} ед.")
print(f"Луна: расстояние {MOON_DISTANCE:.1f} ед., период {MOON_PERIOD} дней")
print(f"TESS: перигей {TESS_PERIGEE:.1f} ед., апогей {TESS_APOGEE:.1f} ед.")
print(f"TESS: период {TESS_PERIOD} дней, эксцентриситет {TESS_e:.3f}")

## Статическая визуализация

In [ ]:
def get_tess_position(t, phase_offset=0):
    """Позиция TESS на эллиптической орбите."""
    # Средняя аномалия
    M = 2 * np.pi * t / TESS_PERIOD + phase_offset
    
    # Решаем уравнение Кеплера (приближённо)
    E = M
    for _ in range(10):
        E = M + TESS_e * np.sin(E)
    
    # Истинная аномалия
    nu = 2 * np.arctan2(np.sqrt(1 + TESS_e) * np.sin(E/2), 
                         np.sqrt(1 - TESS_e) * np.cos(E/2))
    
    # Расстояние от фокуса (Земли)
    r = TESS_a * (1 - TESS_e**2) / (1 + TESS_e * np.cos(nu))
    
    # Координаты (Земля в фокусе)
    x = r * np.cos(nu)
    y = r * np.sin(nu)
    
    return x, y, r

def get_moon_position(t, phase_offset=0):
    """Позиция Луны (круговая орбита)."""
    angle = 2 * np.pi * t / MOON_PERIOD + phase_offset
    x = MOON_DISTANCE * np.cos(angle)
    y = MOON_DISTANCE * np.sin(angle)
    return x, y

# Создаём статический график
fig, ax = plt.subplots(figsize=(12, 12))
ax.set_aspect('equal')
ax.set_facecolor('black')
fig.patch.set_facecolor('black')

# Земля
earth = Circle((0, 0), EARTH_RADIUS, color='dodgerblue', zorder=10)
ax.add_patch(earth)
ax.text(0, -EARTH_RADIUS - 2, 'Земля', color='white', ha='center', fontsize=12)

# Орбита Луны
theta_moon = np.linspace(0, 2*np.pi, 100)
ax.plot(MOON_DISTANCE * np.cos(theta_moon), MOON_DISTANCE * np.sin(theta_moon),
        'gray', ls='--', alpha=0.5, label='Орбита Луны')

# Луна (в одной позиции)
moon_x, moon_y = get_moon_position(0)
moon = Circle((moon_x, moon_y), MOON_RADIUS * 3, color='lightgray', zorder=9)
ax.add_patch(moon)
ax.text(moon_x, moon_y - 3, 'Луна', color='white', ha='center', fontsize=10)

# Орбита TESS (эллипс)
theta_tess = np.linspace(0, 2*np.pi, 200)
tess_orbit_x = []
tess_orbit_y = []
for t in np.linspace(0, TESS_PERIOD, 200):
    x, y, _ = get_tess_position(t)
    tess_orbit_x.append(x)
    tess_orbit_y.append(y)
ax.plot(tess_orbit_x, tess_orbit_y, 'cyan', lw=2, label='Орбита TESS')

# TESS (в одной позиции)
tess_x, tess_y, _ = get_tess_position(0)
ax.plot(tess_x, tess_y, 'r^', markersize=15, zorder=11, label='TESS')

# Апогей и перигей
ax.plot(TESS_APOGEE, 0, 'go', markersize=8)
ax.text(TESS_APOGEE + 1, 0, 'Апогей\n(сбор данных)', color='lime', fontsize=9)

ax.plot(-TESS_PERIGEE, 0, 'yo', markersize=8)
ax.text(-TESS_PERIGEE - 1, 0, 'Перигей\n(передача)', color='yellow', fontsize=9, ha='right')

# Настройки
limit = MOON_DISTANCE + 5
ax.set_xlim(-limit, limit)
ax.set_ylim(-limit, limit)
ax.set_xlabel('x (×10,000 км)', color='white')
ax.set_ylabel('y (×10,000 км)', color='white')
ax.set_title('Орбита TESS и Луны вокруг Земли', color='white', fontsize=14)
ax.legend(loc='upper right', facecolor='black', labelcolor='white')
ax.tick_params(colors='white')
for spine in ax.spines.values():
    spine.set_color('white')

plt.tight_layout()
plt.show()

## Анимация орбиты

In [ ]:
# Создаём анимацию
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_aspect('equal')
ax.set_facecolor('black')
fig.patch.set_facecolor('black')

# Статические элементы
earth = Circle((0, 0), EARTH_RADIUS, color='dodgerblue', zorder=10)
ax.add_patch(earth)

# Орбиты
theta = np.linspace(0, 2*np.pi, 100)
ax.plot(MOON_DISTANCE * np.cos(theta), MOON_DISTANCE * np.sin(theta),
        'gray', ls='--', alpha=0.3)

tess_orbit_x, tess_orbit_y = [], []
for t in np.linspace(0, TESS_PERIOD, 200):
    x, y, _ = get_tess_position(t)
    tess_orbit_x.append(x)
    tess_orbit_y.append(y)
ax.plot(tess_orbit_x, tess_orbit_y, 'cyan', lw=1, alpha=0.5)

# Динамические элементы
moon_patch = Circle((0, 0), MOON_RADIUS * 3, color='lightgray', zorder=9)
ax.add_patch(moon_patch)

tess_marker, = ax.plot([], [], 'r^', markersize=12, zorder=11)
tess_trail, = ax.plot([], [], 'r-', alpha=0.3, lw=1)

# Текст
day_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, color='white',
                   fontsize=12, verticalalignment='top')
distance_text = ax.text(0.02, 0.93, '', transform=ax.transAxes, color='yellow',
                        fontsize=10, verticalalignment='top')
status_text = ax.text(0.02, 0.88, '', transform=ax.transAxes, color='lime',
                      fontsize=10, verticalalignment='top')

limit = MOON_DISTANCE + 5
ax.set_xlim(-limit, limit)
ax.set_ylim(-limit, limit)
ax.set_title('TESS и Луна: резонанс 2:1', color='white', fontsize=14)

# Trail storage
trail_x, trail_y = [], []

def init():
    tess_marker.set_data([], [])
    tess_trail.set_data([], [])
    return tess_marker, tess_trail, moon_patch, day_text, distance_text, status_text

def animate(frame):
    t = frame * 0.5  # 0.5 дня на кадр
    
    # Позиции
    tess_x, tess_y, tess_r = get_tess_position(t)
    moon_x, moon_y = get_moon_position(t)
    
    # Обновляем TESS
    tess_marker.set_data([tess_x], [tess_y])
    
    # Trail
    trail_x.append(tess_x)
    trail_y.append(tess_y)
    if len(trail_x) > 50:
        trail_x.pop(0)
        trail_y.pop(0)
    tess_trail.set_data(trail_x, trail_y)
    
    # Обновляем Луну
    moon_patch.center = (moon_x, moon_y)
    
    # Расстояние TESS-Луна
    dist_to_moon = np.sqrt((tess_x - moon_x)**2 + (tess_y - moon_y)**2) * SCALE
    
    # Текст
    day_text.set_text(f'День: {t:.1f}')
    distance_text.set_text(f'TESS до Луны: {dist_to_moon:,.0f} км')
    
    # Статус
    if tess_r > TESS_a:  # Около апогея
        status_text.set_text('📡 Сбор данных (апогей)')
        status_text.set_color('lime')
    else:
        status_text.set_text('📤 Передача данных')
        status_text.set_color('yellow')
    
    # Предупреждение о близости к Луне
    if dist_to_moon < 100000:  # < 100,000 км
        status_text.set_text('⚠️ БЛИЗКО К ЛУНЕ - возможен артефакт!')
        status_text.set_color('red')
    
    return tess_marker, tess_trail, moon_patch, day_text, distance_text, status_text

# Создаём анимацию (2 полных периода Луны = ~55 дней)
n_frames = 110
anim = FuncAnimation(fig, animate, init_func=init, frames=n_frames,
                     interval=100, blit=True)

plt.close()  # Не показываем статичный кадр
HTML(anim.to_jshtml())

## Когда возникают проблемы?

Анализируем моменты сближения TESS и Луны.

In [ ]:
# Симулируем 1 год орбиты
days = np.linspace(0, 365, 10000)

distances = []
for t in days:
    tess_x, tess_y, _ = get_tess_position(t)
    moon_x, moon_y = get_moon_position(t)
    dist = np.sqrt((tess_x - moon_x)**2 + (tess_y - moon_y)**2) * SCALE
    distances.append(dist)

distances = np.array(distances)

# График расстояния
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Расстояние во времени
ax1 = axes[0]
ax1.plot(days, distances / 1000, 'b-', lw=0.5)
ax1.axhline(100, color='red', ls='--', label='Опасная зона (100,000 км)')
ax1.fill_between(days, 0, 100, where=distances/1000 < 100, alpha=0.3, color='red')
ax1.set_xlabel('День года')
ax1.set_ylabel('Расстояние TESS-Луна (тыс. км)')
ax1.set_title('Расстояние между TESS и Луной')
ax1.legend()
ax1.set_ylim(0, 800)

# Гистограмма
ax2 = axes[1]
ax2.hist(distances / 1000, bins=50, edgecolor='black')
ax2.axvline(100, color='red', ls='--', lw=2, label='Опасная зона')
ax2.set_xlabel('Расстояние TESS-Луна (тыс. км)')
ax2.set_ylabel('Частота')
ax2.set_title('Распределение расстояний')
ax2.legend()

plt.tight_layout()

# Статистика
close_approaches = np.sum(distances < 100000)
pct_danger = close_approaches / len(distances) * 100
print(f"\nСтатистика за год:")
print(f"  Минимальное расстояние: {distances.min():,.0f} км")
print(f"  Среднее расстояние: {distances.mean():,.0f} км")
print(f"  Время в опасной зоне (<100,000 км): {pct_danger:.1f}%")

## Связь с нашими данными

Сектор 70 наблюдался в **сентябре-октябре 2023**. Посмотрим, когда Луна была близко.

In [ ]:
# Сектор 70: BTJD 3208-3234 (примерно 26 дней)
# BTJD = BJD - 2457000
# Артефакт был в BTJD 3215-3221

sector_start = 0  # относительное начало
sector_end = 26   # 26 дней наблюдений
artifact_start = 7  # BTJD 3215 - 3208 = день 7
artifact_end = 13   # BTJD 3221 - 3208 = день 13

# Симулируем этот период
sector_days = np.linspace(sector_start, sector_end, 1000)
sector_distances = []

for t in sector_days:
    tess_x, tess_y, _ = get_tess_position(t)
    moon_x, moon_y = get_moon_position(t)
    dist = np.sqrt((tess_x - moon_x)**2 + (tess_y - moon_y)**2) * SCALE
    sector_distances.append(dist)

sector_distances = np.array(sector_distances)

# График
fig, ax = plt.subplots(figsize=(14, 5))

ax.plot(sector_days, sector_distances / 1000, 'b-', lw=2, label='Расстояние TESS-Луна')
ax.axhline(100, color='red', ls='--', alpha=0.7)

# Отмечаем период артефакта
ax.axvspan(artifact_start, artifact_end, alpha=0.3, color='red', 
           label=f'Артефакт (дни {artifact_start}-{artifact_end})')

ax.set_xlabel('День сектора 70')
ax.set_ylabel('Расстояние TESS-Луна (тыс. км)')
ax.set_title('Сектор 70: Расстояние до Луны и период артефакта')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()

# Найти минимум в период артефакта
artifact_mask = (sector_days >= artifact_start) & (sector_days <= artifact_end)
min_dist_artifact = sector_distances[artifact_mask].min()
print(f"\nМинимальное расстояние в период артефакта: {min_dist_artifact:,.0f} км")

## Итоги

### Орбита TESS:
- Эллиптическая орбита в резонансе 2:1 с Луной
- Период: 13.7 дней
- Данные собираются около апогея (далеко от Земли)

### Проблемы с Луной:
- Когда TESS проходит близко к Луне, свет от неё засвечивает камеры
- Это происходит регулярно из-за резонанса орбит
- В Секторе 70 это было в дни 7-13 (BTJD 3215-3221)

### Практический вывод:
При анализе данных TESS всегда проверяйте Data Release Notes на сайте MAST — там указаны периоды с проблемами качества.